# Data Source
Loading, cleaning, and pre-processing
- acc_user
- acc_category, acc_category_income
- acc_cashflow, the fact table created from expense and income tables:
    - acc_user_detail, acc_group_detail, acc_user_detail_income, acc_group_detail_income

_to-do:_ check frequency calculations  
_to-do:_ count the number of _is_agree_ members connected to each user

In [ ]:
import os
from sqlalchemy import create_engine
import pandas as pd

user = os.getenv("MYSQL_USER")
password = os.getenv("MYSQL_PASSWORD")
host = "localhost"
database = "checkchick"

engine = create_engine(f"mysql+pymysql://{user}:{password}@{host}/{database}")

## Users
`user_id`  
`is_bot`  
`is_agree`: is communication from LINE OA blocked by the user  
`ts`: timestamp when the record was created but unclear if it is updated later

In [ ]:
query = """
SELECT
  user_id,
  isBot is_bot,
  isAgree is_agree,
  min(CREDTM) ts
FROM ACC_USER
WHERE LENGTH(TRIM(GROUP_ID)) = 0
GROUP BY user_id, is_bot, is_agree
"""

users = pd.read_sql(query, con=engine,
                    dtype=({'is_bot':'bool', 'is_agree':'bool', 'ts':'datetime64[ns]'}))
# users.info()

In [ ]:
# check for duplicates
x = users.groupby('user_id').size()
dups=x[x > 1].index
len(dups)

In [ ]:
users[users.user_id.isin(dups)].shape

In [ ]:
# what is dup ratio?
89192/44596

In [ ]:
users[users.user_id.isin(dups)].sort_values(['user_id', 'ts']).tail(10)

Remove duplicates by keeping the last row (most recent) in each group.


In [ ]:
users.sort_values(['user_id','ts'], inplace=True)

In [ ]:
x = users.groupby('user_id', as_index=False).last()
x[x.user_id.isin(dups)]

In [ ]:
x.shape

In [ ]:
users = x

In [ ]:
# how many Bots?
print(users.is_bot.sum())

In [ ]:
users.query("is_bot")

Impute `user_id`  
Real user_id starts with captial 'U'

In [ ]:
i = users.user_id.str.lower().str.startswith('u')
users[i].shape
users.loc[i, 'user_id'] = users[i]['user_id'].str.capitalize()

Cohorts by user timestamp
_n.b._, this is not always the time user followed or added the OA

In [ ]:
cohorts = users.groupby(users.ts.dt.year).size()
cohorts.name = 'n_new_user'
_ = cohorts.plot.bar(rot=0, xlabel='cohort year', ylabel='users')

In [ ]:
# user count vs. block count
print((~users.is_bot).sum(), (~users.is_bot & ~users.is_agree).sum())

In [ ]:
users.to_feather('../data/users.feather')

## Groups
group_id:user_id relationship table

In [ ]:
query = """
SELECT DISTINCT
  group_id,
  user_id
FROM ACC_USER
WHERE LENGTH(TRIM(GROUP_ID)) > 0;
--    AND isBot IS FALSE
"""

groups = pd.read_sql(query, con=engine)
# groups.info()

In [ ]:
groups.to_feather('../data/groups.feather')

## Members and Connections
For each user, 
identify connected members by
self-joining _groups_ on user_id to compute
number of distinct users interacted within groups

In [ ]:
# user's groups and their associated users (members),
# i.e., user's connections with other users thru cooperative bookkeeping 

mbr = groups.merge(users[['user_id', 'is_agree']])

mbr = mbr[['user_id', 'is_agree', 'group_id']] \
    .merge(mbr[['group_id', 'user_id', 'is_agree']],
           on='group_id', how='left', suffixes=('', '_mbr'))
# mbr.columns = ['user_id', 'user_is_agree', 'group_id', 'member_id', 'mbr_is_agree']
mbr['self'] = mbr.user_id == mbr.user_id_mbr
mbr.info()

In [ ]:
mbr.to_feather('../data/members.feather')

In [ ]:
# count the participants at each group level for every user

x = mbr.groupby(['user_id', 'group_id']) \
    .agg({'user_id_mbr':'nunique'}) \
    .reset_index(1) \
    .rename(columns={'user_id_mbr':'nbr_member'})
# x.describe()

_ = x.plot.hist(bins=50, alpha=0.7)

_**n.b.:**_ Groups with only one participant should be excluded in the stat summary.

In [ ]:
# for every user, count unique users across _all_ associated groups

x = mbr.groupby('user_id').agg({'group_id': 'nunique', 'user_id_mbr':'nunique'})
x.columns = ['n_grp', 'n_connection']
x['n_connection'] = x['n_connection'] - 1 # remove user itself from count
# _ = cnx.hist(bins=50, grid=False, alpha=.7)
x.describe()

In [ ]:
# how many connections have not blocked/unfollowed
agreed = mbr.groupby('user_id')[['is_agree_mbr', 'user_id_mbr', 'self']] \
    .apply(lambda x: x.loc[~x.self & x.is_agree_mbr, 'user_id_mbr'].nunique()).rename('n_agree')
cnx = x.merge(agreed, on='user_id', how='left')

In [ ]:
cnx.describe()

In [ ]:
# cnx.query("user_id=='Ufff1d2f36052bfa45e1cc71ab0bff020'")

In [ ]:
# mbr[mbr.user_id=='Ufff1d2f36052bfa45e1cc71ab0bff020']

## Categories

In [ ]:
query = """
SELECT
  id category_id,
  category,
  created_at ts_create,
  updated_at ts_update
FRoM zoo_checkchick3.ACC_CATEGORY
"""

a = pd.read_sql(query, con=engine)
a['is_expense'] = True
# a.info()

query = """
SELECT
  id category_id,
  category,
  created_at ts_create,
  updated_at ts_update
FRoM zoo_checkchick3.ACC_CATEGORY_INCOME
"""

b = pd.read_sql(query, con=engine)
b['is_expense'] = False
# b.info()

# union and ensure index is reset
category = pd.concat([a, b], ignore_index=True).convert_dtypes()
# category.info()
category.groupby('is_expense').size()

## Cash Flow

In [ ]:
# acc_cashflow dataset
# check number of records (expense and income entries) each month

cashflow = pd.read_feather('../data/cashflow.feather')

df = cashflow \
    .groupby(cashflow.ts.dt.to_period('M')) \
    .agg(total = ('ts', 'size'),
         nbr_expense_entry = ('is_expense', 'sum'),
         nbr_group_entry = ('is_group', 'sum')
        )
_ = df.plot.line(y=['total', 'nbr_expense_entry', 'nbr_group_entry'], xlabel='')
print(df.describe())

In [ ]:
# print(df[df.total > df.total.quantile(.51)])

__*Observation:*__
- errorenous timestamps
- contrary to my expecation, personal entries makes up significally larger portions of records
- expenses makes up larger portion of the records; this aligns with the norm for personal finance dataset

Let `tsl` be the observation time period

In [ ]:
# select date range
# between '2018-06-01' AND '2025-06-01'
# to exclude erroneous rows

tsl = pd.to_datetime(['2018-06-01', '2025-06-01'])
cashflow = cashflow.query("@tsl[0] <= ts < @tsl[1]").copy()
cashflow.info()

_**n.b.,**_ __amt__ is float64. _I expect this to be whole number._

In [ ]:
cashflow.isna().sum()

In [ ]:
print(cashflow.group_id.count() / cashflow.shape[0])

_**Observation:**_ group entries make up about 15.8% of the records.

_**Question:**_ Should factional amounts be rounded up?

In [ ]:
cashflow.query("amt % 1 != 0").groupby('is_expense')['amt'].describe()

In [ ]:
x = cashflow.loc[:, ['is_expense', 'amt']]
x['amt_rounded'] = round(x['amt'])
x['amt_diff'] = x['amt'] - x['amt_rounded']
x.groupby('is_expense').agg({'is_expense':'size', 'amt':'sum', 'amt_rounded':'sum', 'amt_diff':'sum'})

In [ ]:
# impute fractional amounts by rounding up if amt > 0 and down if amt < 0

cashflow['amt'] = cashflow['amt'].round().astype('Int64')

In [ ]:
cashflow.info()

In [ ]:
# Should amount equals 0 be excluded?

cashflow[cashflow.amt == 0].count()

In [ ]:
# check number of active users in each month, i.e. period

cashflow['yyyy_mm'] = cashflow['ts'].dt.to_period('M')
cashflow['yyyy'] = cashflow['ts'].dt.year
cashflow['mm'] = cashflow['ts'].dt.month
cashflow['wk'] = cashflow['ts'].dt.isocalendar().week

#cashflow.groupby(['yyyy', 'mm']).agg({'user_id':'nunique'}).unstack(level=1)

In [ ]:
_ = cashflow.groupby('yyyy_mm').agg({'user_id':'nunique'}) \
    .plot(y='user_id', kind='line', figsize=(12, 6), title="User Count by Month")

__*Observation:*__ number of active users gradually declined but not obvious

In [ ]:
cashflow.describe()

_**Oberservation:**_ expense records out weight income records 3:1

Impute `group_id` is blank but not `None`

In [ ]:
bad = cashflow.group_id.str.strip().str.len() == 0  # group_id missing
bad.value_counts()

In [ ]:
cashflow.loc[bad, 'user_id'].nunique()

In [ ]:
# impute
cashflow.loc[bad, ['group_id', 'is_group']] = [None, False]

Erroneous `user_id`  
Valid `user_id` must start with captial U

In [ ]:
cashflow['isBad'] = ~cashflow.user_id.str.lower().str.startswith('u')

In [ ]:
cashflow.groupby('isBad').size()

In [ ]:
cashflow[bad].describe()

Impute invalid `user_id`  
Make sure they start with capital U

In [ ]:
cashflow.loc[~cashflow.isBad, 'user_id'] = cashflow[~cashflow.isBad]['user_id'].str.capitalize()

In [ ]:
#save
cashflow.to_feather('../data/cashflow_cleaned.feather')

# Tidy Dataset

Let the final tidy dataset be `td`. Proceed with the data preparation as follow:

## Bookkeepers: basic stats

Based on acc_cashflow,
`tenure` is defined as to begin when user submits first entry,
and does not consider when a user first follows or adds LINE OA
(offical account) since that timestamp is not recorded.

`days_active` is the number of unique days that a user log any transaction.

Transaction amounts 0 (zero) are dismissed.

Transactions with invalid user_id are dismissed.

In [ ]:
# user tenure, income and expense entry stats,
# including group entries
# excluding amount = 0

bkp = cashflow[(cashflow.amt != 0) & ~cashflow.isBad] \
    .groupby('user_id') \
    .agg(tenure = ('ts', lambda x: x.max() - x.min()),
         days_active = ('ts', lambda x: x.dt.date.nunique()),
         pct_active = ('ts', 'min'), # place holder
         first_entry = ('ts', 'min'),
         last_entry = ('ts', 'max'),
         nbr_entry = ('user_id', 'count'),
         total_exp = ('amt', lambda x: x[x < 0].sum()),
         nbr_exp = ('amt', lambda x: x[x < 0].count()),
         total_inc = ('amt', lambda x: x[x > 0].sum()),
         nbr_inc = ('amt', lambda x: x[x > 0].count())
        )

In [ ]:
#tenure.days_active / 
# x = (tenure.last_entry.dt.floor('d') - tenure.first_entry.dt.floor('d'))
# x = (tenure.last_entry.dt.date - tenure.first_entry.dt.date)
y = (bkp.last_entry.dt.floor('d') - bkp.first_entry.dt.floor('d')).dt.days + 1
bkp['pct_active'] = round(bkp.days_active / y * 100, 2)

In [ ]:
bkp.tenure.describe()

In [ ]:
# users who's been active for only 1 day but made more than 5 entries
# name this type of users
_ = bkp.loc[(bkp.days_active == 1) & (bkp.nbr_entry > 5), 'nbr_entry'].hist()

In [ ]:
bkp.describe(percentiles=[.25, .5, .75, .9, .95, .97, .99])

In [ ]:
tsl[1] - pd.to_datetime('2025-04-03 08:00')

_**Initial observation:**_ Of the 400,831 users, 97% of which had **not** logged entries since 2024-12-08 (in the last 58 days

In [ ]:
# to-do: churned vs non-churned
bkp['duration'] = bkp.tenure.dt.total_seconds() / 24 / 3600
_ = bkp.plot(kind='scatter', x='duration', y='pct_active', alpha=.3)

In [ ]:
_ = bkp.plot(kind='scatter', x='duration', y='days_active', alpha=.3)

In [ ]:
bkp.drop(columns='duration', inplace=True)

## WIP Transaction Category 

In [ ]:
# WIP category count
td_cat = cashflow[~cashflow.isBad].groupby(['user_id', 'is_expense'])['category_id'].nunique().unstack(level=1, fill_value=0)
td_cat.describe()

In [ ]:
cashflow[~cashflow.isBad & ~cashflow.is_expense].groupby(['user_id'])['category_id'].nunique().describe(percentiles=[.25, .5, .54, .55, .8, .9, .95])

In [ ]:
cashflow[~cashflow.isBad & cashflow.is_expense].groupby(['user_id'])['category_id'].nunique().describe(percentiles=[.25, .60, .70, .75, .8, .85, .9, .95])

_**Initial observation:**_ 
- 50% or more did not track income
- 54% of who did track used one cateogry
- 60% of who tracked expenses utilized 3 or less categories

## Group Bookkeepers

_to-do:_ add count of categories

In [ ]:
# user group expense and income entry stat per user

td_grp = cashflow[cashflow.is_group & (cashflow.amt != 0) & ~cashflow.isBad] \
    .groupby('user_id') \
    .agg(n_grp = ('group_id', 'nunique'),
         first_grp_entry = ('ts', 'min'),
         last_grp_entry = ('ts', 'max'),
         nbr_grp_exp = ('amt', lambda x: x[x < 0].count()),
         grp_exp = ('amt', lambda x: x[x < 0].sum()),
         nbr_grp_inc = ('amt', lambda x: x[x > 0].count()),
         grp_inc = ('amt', lambda x: x[x > 0].sum())
        )
# td_grp.info()

In [ ]:
td_grp.describe()

## Frequency

What is the typical frequency of logging financial transactions?  
- number of times per week
- interval (or elapsed time) between events (logging transactions)

In [ ]:
# set datetime index in order to resample frequency of event
# cashflow.reset_index(inplace=True)
cashflow = cashflow.set_index('ts').sort_index()

In [ ]:
weekly_counts = cashflow[~cashflow.isBad].groupby(['user_id', pd.Grouper(freq='W')])['user_id'].size()

In [ ]:
fq = weekly_counts.groupby('user_id').agg({'count', 'median', 'mean'})
fq.columns = ['nbr_wks', 'fq_median', 'fq_mean']
fq.info()

In [ ]:
fq.describe(percentiles=[.25, .5, .6, .7, .75, .8, .9, .95, .99])

calcuate interval

In [ ]:
# if we just want to know how frequent users records their personal finance,
# it is not important to separate income from expense entries

cashflow.reset_index(inplace=True)
y_sorted = cashflow[~cashflow.isBad].sort_values(['user_id', 'ts'])
y_sorted['time_elapsed'] = y_sorted.groupby('user_id')['ts'].diff()

In [ ]:
# y_sorted[['user_id', 'ts', 'time_elapsed']].tail(30)

In [ ]:
# intervals = y_sorted.groupby('user_id').agg({'days_elapsed':['median', 'mean', 'max']})
# n.b. a known bug with median, use quantile(0.5) workaround
#      workaround also includes renaming the columns <lambda_0>
intervals = y_sorted.groupby('user_id').agg({'time_elapsed':[lambda x: x.quantile(0.5), 'mean', 'max']})
intervals.columns = intervals.columns.set_levels(['median', 'mean', 'max'], level=1)
intervals.describe()

In [ ]:
# flatten multilevel column index
# intervals.columns = intervals.columns.to_flat_index()
intervals.columns = ['_'.join(col) for col in intervals.columns]

In [ ]:
# intervals.head(30)

In [ ]:
fq.info()

In [ ]:
intervals.info()

In [ ]:
freq = fq.merge(intervals, on='user_id').convert_dtypes()

In [ ]:
freq.info()

### WIP

In [ ]:
# 
x_sorted = cashflow.sort_values(['user_id', 'is_expense', 'ts'])
x_sorted['days_elapsed'] = x_sorted.groupby(['user_id', 'is_expense'])['ts'].diff()

In [ ]:
x_sorted[['user_id', 'is_expense', 'ts', 'days_elapsed']].tail(30)

In [ ]:
fq_ = x_sorted.groupby(['user_id', 'is_expense']).agg({'days_elapsed':[lambda x: x.quantile(0.5), 'mean', 'max']})

In [ ]:
fq_.query("is_expense").describe()

In [ ]:
#fq =
fq_.groupby(['is_expense']).agg(['min', 'median', 'mean', 'max'])
# fq

In [ ]:
fq_.xs('Uffffed94576a41cb306b899c40719ed9', level='user_id')

In [ ]:
fq_.xs(False, level='is_expense').agg(['min','mean','median'])
# fq.index

## Merge and Impute

In [ ]:
td = users[['user_id', 'is_agree', 'ts']].rename(columns={'ts':'user_ts'}) \
    .merge(bkp, on='user_id', how='right') \
    .merge(freq, on='user_id', how='left') \
    .merge(cnx, on='user_id', how='left') \
    .merge(td_grp, on='user_id', how='left', suffixes=('', '_activated')) \
    .convert_dtypes()
# td.info()

In [ ]:
values = {'is_agree':False,
          'n_grp':0, 'n_connection':0, 'n_agree':0,
          'n_grp_activated':0,
          'grp_exp':0, 'nbr_grp_exp':0,
          'grp_inc':0, 'nbr_grp_inc':0}
td.fillna(value=values, inplace=True)

In [ ]:
td.describe()

In [ ]:
td.info()

In [ ]:
# raise exception if file already exists but allows the run to proceed
# to-do: convert user tenure from timedelta days to int before .to_csv
td.to_feather('../data/tidy.feather')
# td.drop(columns=['user_ts']).to_csv('../reports/tidy.csv', mode='x')

In [ ]:
# interesting example
td[td.user_id=='Uffffed94576a41cb306b899c40719ed9']

In [ ]:
# which users has big gaps between logging
td[td.time_elapsed_max.dt.days > 106].groupby('is_agree')[['days_active', 'tenure', 'nbr_entry']].median()

# Supplemental

```sql
-- consolidate expense and income entry to simplify analysis

USE checkchick;

DROP TABLE IF EXISTS acc_cashflow;

CREATE TABLE acc_cashflow (
  user_id VARCHAR(100) NOT NULL,
  ts DATETIME NOT NULL,
  amt DOUBLE NOT NULL,
  is_expense BOOLEAN,
  is_group BOOLEAN,
  group_id VARCHAR(100),
  category_id INT,
  note VARCHAR(160)
) COMMENT = 'consolidated expense and income entries'
;

-- insert personal expense
INSERT INTO acc_cashflow
SELECT
  USER_ID user_id,
  CREDTM ts,
  -AMOUNT amt,
  TRUE is_expense,
  FALSE is_group,
  NULL group_id,
  CATEGORY category_id,
  NOTE note
FROM ACC_USER_DETAIL
;

-- insert group expense
INSERT INTO acc_cashflow
SELECT
  USER_ID user_id,
  CREDTM ts,
  -AMOUNT amt,
  TRUE is_expense,
  TRUE is_group,
  GROUP_ID group_id,
  CATEGORY category_id,
  NOTE note
FROM ACC_GROUP_DETAIL
;

-- insert personal income
INSERT INTO acc_cashflow
SELECT
  USER_ID user_id,
  CREDTM ts,
  AMOUNT amt,
  FALSE is_expense,
  FALSE is_group,
  NULL group_id,
  CATEGORY category_id,
  NOTE note
FROM ACC_USER_DETAIL_INCOME
;

-- insert group income
INSERT INTO acc_cashflow
SELECT
  USER_ID user_id,
  CREDTM ts,
  AMOUNT amt,
  FALSE is_expense,
  TRUE is_group,
  GROUP_ID group_id,
  CATEGORY category_id,
  NOTE note
FROM ACC_GROUP_DETAIL_INCOME
;```